## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images, and extract the object points and images points

## Compute camera calibration and distortion correction for chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nx=9
ny=6
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        #ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

## Undistort and unwarp the images using camera calibration and perspective transform for Chessboards 

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nx=9
ny=6
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')
img=cv2.imread(images[19])
img=np.array(img)

## Define the function for image unwarp
def Unwarp_img(img, nx, ny):
        # Find the chessboard corners
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners=cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

        undist = cv2.undistort(img, mtx, dist, None, mtx)
        offset=100 
        img_size=(gray.shape[1],gray.shape[0])
        src=np.float32([corners[0,:,:], corners[nx-1,:,:],corners[-1,:,:],corners[-nx,:,:]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [img_size[0]-offset, img_size[1]-offset], 
                                     [offset, img_size[1]-offset]])
        # Given src and dst points, calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        Minv = cv2.getPerspectiveTransform(dst, src)
        # Warp the image using OpenCV warpPerspective()
        warped = cv2.warpPerspective(undist, M, img_size)
    return warped, M

warped_img, perspective_M=Unwarp_img(img, nx, ny)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(warped_img)
ax2.set_title('Undistorted and Warped Image', fontsize=30)
plt.savefig('Undistorted_Warped_Image.png')


In [ ]:
thresh_grad_min = 50
thresh_grad_max = 200
color_thre_min, color_thre_max=130, 255
## Define Color & gradients function
def color_threshold(img, thresh_grad_min, thresh_grad_max):
    hls=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel=hls[:,:,2]
    h_channel=hls[:,:,0]
    l_channel=hls[:,:,1]
    gray=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
     
    ## Sobelx : Apply sobelx to s_channel
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_grad_min) & (scaled_sobel <= thresh_grad_max)] = 1
    
    # Color threshold
    s_binary=np.zeros_like(s_channel)
    s_binary[(s_channel>=color_thre_min)&(s_channel<color_thre_max)]=1
    # stack each channel to an image to see the contribution of individual channels
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    # combine two binary thresholds
    combined_binary=np.zeros_like(s_binary)
    combined_binary[(sxbinary==1)|(s_binary==1)]=1
    return  combined_binary
out=color_threshold(img_warped, thresh_grad_min, thresh_grad_max)


f, (ax1, ax2)=plt.subplots(1,2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img_warped)
ax1.set_title('warped Image', fontsize=30)
ax2.imshow(out, cmap='gray')
ax2.set_title('gradient threshold', fontsize=30)
  


# Pipeline for Lane detection on images

### Define all functions



## Apply Camera calibration, distortion correction, color&gradients threshold and perspective transforms to the raw testing images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline
## Apply Camera calibration, distortion correction
def camera_calibration():
    nx=9
    ny=6
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
    images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
    for fname in images:
        img_chess = cv2.imread(fname)
        gray_chess = cv2.cvtColor(img_chess,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray_chess, (9,6),None)
    

    # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    shape=gray_chess.shape[::-1]
    np.save('objpoints', objpoints)
    np.save('imgpoints', imgpoints)
    np.save('shape', shape)
    
    return None
## Apply Camera calibration, distortion correction
def undistortion(img):
# Arrays to store object points and image points from all the images.
    objpoints = np.load('objpoints.npy') # 3d points in real world space
    imgpoints = np.load('imgpoints.npy') # 2d points in image plane.
    shape=tuple(np.load('shape.npy'))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints,shape,None, None)
    undist=cv2.undistort(img, mtx, dist, None, mtx)
    return undist  

# read an image from a folder
#img=mpimg.imread('solidYellowCurve2.jpg')
img=cv2.imread('../test_images/test5.jpg')
b,g,r=cv2.split(img)
img=cv2.merge([r,g,b])
original_img=img
# print out some info about this image
#img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print ('This image is:', type(img),'with dimensions of',img.shape)

# transform from color to gray scale
img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY) #mpimg loads image as RGB
dst=undistortion(img)
f, (ax1, ax2)=plt.subplots(1,2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)
  

In [ ]:
## Perspective transform
## Define the function for image unwarp
def warped_img(img):
    gray=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_size=(gray.shape[1],gray.shape[0])
    src=np.float32([[580,460],[700,460],[1040,680],[260,680]])
    #src=np.float32([[580,460],[700,460],[1200,720],[200,720]])
    dst = np.array([[200, 0], [1000, 0], [1000, img.shape[0]], [200, img.shape[0]]], np.float32)
# Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
# Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size)
    return warped, M, Minv
img_warped, M, Minv=warped_img(img)

f, (ax1, ax2)=plt.subplots(1,2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(img_warped)
ax2.set_title('Warped image', fontsize=30)
    

In [ ]:
thresh_grad_min = 20
thresh_grad_max = 200
## Define Color & gradients function
def color_threshold(img, thresh_grad_min, thresh_grad_max):
    hls=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel=hls[:,:,2]
    h_channel=hls[:,:,0]
    l_channel=hls[:,:,1]
    gray=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hsv=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    ## Sobelx
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    
    ## Sobely
    sobely = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1) # Take the derivative in y
    abs_sobely = np.absolute(sobely) # Absolute x derivative to accentuate lines away from horizontal
    mag=np.sqrt(sobelx**2+sobely**2)
    scaled_sobel = np.uint8(255*mag/np.max(mag))
    # Threshold xy gradient
    sxybinary = np.zeros_like(scaled_sobel)
    sxybinary[(scaled_sobel >= thresh_grad_min) & (scaled_sobel <= thresh_grad_max)] = 1
    
    # detecting yellows using hsv color space
    yellow_low=np.array([0,100,100], np.uint8)
    yellow_high=np.array([50, 255, 255], np.uint8)
    mask_yellow=cv2.inRange(hsv, yellow_low, yellow_high)
    
    # detecting white using hsv color space
    white_low=np.array([20, 0, 200], np.uint8)
    white_high=np.array([255, 80, 255], np.uint8)
    mask_white=cv2.inRange(hsv, white_low, white_high)
    mask=cv2.bitwise_or(mask_yellow, mask_white)
    
    # detecting edge using color threshold in s_channel
    #color_thre_min=145
    #color_thre_max=200
    #s_binary=np.zeros_like(s_channel)
    #s_binary[(s_channel>=color_thre_min)&(s_channel<color_thre_max)]=1
    # detecting shadowness and brightness change
    color_thre_min=145
    color_thre_max=200
    lab=cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    b_channel=lab[:,:,2]
    b_binary=np.zeros_like(b_channel)
    b_binary[(b_channel>=color_thre_min)&(s_channel<color_thre_max)]=1
    
    
    # stack each channel to an image to see the contribution of individual channels
    #color_binary = np.dstack(( np.zeros_like(sxybinary), sxybinary, mask)) * 255
    # combine three binary thresholds
    combined_binary=np.zeros_like(sxybinary)
    combined_binary[(sxybinary==1)|(mask==1)|(b_binary==1)]=1
    return  combined_binary

out=color_threshold(img_warped, thresh_grad_min, thresh_grad_max)

f, (ax1, ax2)=plt.subplots(1,2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img_warped)
ax1.set_title('warped Image', fontsize=30)
ax2.imshow(out, cmap='gray')
ax2.set_title('gradient threshold', fontsize=30)
  

In [ ]:

## Define the function of detecting lane lines

def find_lines(binary):
    # calculate the current base leftx and rightx pixels location
    image_height=binary.shape[0]
    image_width=binary.shape[1]
    histogram=np.sum(binary[binary.shape[0]//2:,:], axis=0)
    midpoint=np.int(binary.shape[1]//2)
    leftx_base=np.argmax(histogram[:midpoint])           # define the base left lanes location
    rightx_base=np.argmax(histogram[midpoint:])+midpoint  # define the base right lanes location
    leftx_current=leftx_base
    rightx_current=rightx_base
    out_img=np.dstack((binary, binary, binary))
    # nonzero x & y locations
    nonzerox=np.array(binary.nonzero()[1])
    nonzeroy=np.array(binary.nonzero()[0])
    # left and right lane lines indices
    leftx_ind, rightx_ind=[],[]
    # Hyper parameters for windows
    nwindow=9
    margin=100
    minpxl=50
    
    for window in range(nwindow):
        # define the parameters of windwos on image
        win_height=np.int(image_height//nwindow)
        win_y_low=image_height-(window+1)*win_height
        win_y_high=image_height-window*win_height
        win_left_low=leftx_current-margin
        win_left_high=leftx_current+margin
        win_right_low=rightx_current-margin
        win_right_high=rightx_current+margin
        
        # identify the nonzero pixels within the window
        detected_leftx_ind=((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_left_low) &  (nonzerox < win_left_high)).nonzero()[0]
        
        detected_rightx_ind=((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_right_low) &  (nonzerox < win_right_high)).nonzero()[0]
        
        if len(detected_leftx_ind)>minpxl:
            leftx_current=np.int(np.mean(nonzerox[detected_leftx_ind]))
        if len(detected_rightx_ind)>minpxl:
            rightx_current=np.int(np.mean(nonzerox[detected_rightx_ind]))
        leftx_ind.append(detected_leftx_ind)
        rightx_ind.append(detected_rightx_ind)
    try:
        leftx_ind=np.concatenate(leftx_ind)
        rightx_ind=np.concatenate(rightx_ind)
    except ValueErrors:
        
        pass
    
    leftx=nonzerox[leftx_ind]
    rightx=nonzerox[rightx_ind]
    lefty=nonzeroy[leftx_ind]
    righty=nonzeroy[rightx_ind]
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary.shape[0]-1, binary.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    return left_fitx, right_fitx, ploty

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# define polynomial function of lane lines

def polynomial_fit(binary, Minv, left_fitx, right_fitx, ploty):

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (binary.shape[1], binary.shape[0])) 
    

    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]
    #out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    return newwarp

def curvature_cal(binary):
    # Find our lane pixels first
    leftx, rightx, ploty = find_lines(binary)
    # covert x, y from pixels to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    image_height=binary.shape[0]
    image_width=binary.shape[1]
    histogram=np.sum(binary[binary.shape[0]//2:,:], axis=0)
    midpoint=np.int(binary.shape[1]//2)
    leftx_base=np.argmax(histogram[:midpoint])           # define the base left lanes location
    rightx_base=np.argmax(histogram[midpoint:])+midpoint  # define the base right lanes location
    lane_center=np.int((leftx_base+rightx_base)//2)
    img_center=np.int(image_width//2)
    Veh_pos=np.float32((img_center-lane_center)*xm_per_pix)  # calculate the vehicel position, left '-', right off center '+'
    
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)

    # Generate x and y values for plotting
    y_eval=np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    lane_curverad=(left_curverad+right_curverad)/2

    return lane_curverad, Veh_pos

warped_binary=color_threshold(img_warped, thresh_grad_min, thresh_grad_max)

f, (ax1, ax2)=plt.subplots(1,2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img_warped)
ax1.set_title('Warped Image', fontsize=30)
ax2.imshow(warped_binary, cmap='gray')
ax2.set_title('warped binary', fontsize=30)

    
    

In [ ]:

def process_image(img):
    # print out some info about this image
    #img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # transform from color to gray scale
    #b,g,r=cv2.split(img)
    #img=cv2.merge([r,g,b])
    img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY) #mpimg loads image as RGB
    # Apply Gaussian smoothing 
    #kernel_size=5
    #blur_gray=cv2.GaussianBlur(img_gray,(kernel_size,kernel_size),0,0)
    # Undistort the image

    undistort_img=undistortion(img)
    #color & gradients threshold
    img_warped, perspective_M, Minv=warped_img(undistort_img)
    warped_binary=color_threshold(img_warped, thresh_grad_min, thresh_grad_max)
    vertices = np.array([[(120, 710),(120, 0), (1150, 0), (1200,710)]], dtype=np.int32)
    masked_binary = region_of_interest(warped_binary, vertices)
    left_fitx, right_fitx, ploty=find_lines(masked_binary)
    
    global l_fit_buffer
    global r_fit_buffer
    global old_img_lines

    if old_img_lines is None:
        old_img_lines = masked_binary

    ret = cv2.matchShapes(old_img_lines, masked_binary, 1, 0.0)
    print(ret)
    if ret < 35:
        old_img_lines = masked_binary

        if l_fit_buffer is None:
            l_fit_buffer = np.array([left_fitx])

        if r_fit_buffer is None:
            r_fit_buffer = np.array([right_fitx])

        l_fit_buffer = np.append(l_fit_buffer, [left_fitx], axis=0)[-FILTER_SIZE:]
        r_fit_buffer = np.append(r_fit_buffer, [right_fitx], axis=0)[-FILTER_SIZE:]
        l_fit_mean=np.mean(l_fit_buffer[-15:], axis=0)
        r_fit_mean=np.mean(r_fit_buffer[-15:], axis=0)
    else:
        # Compute the mean
        l_fit_mean = np.mean(l_fit_buffer, axis=0)
        r_fit_mean = np.mean(r_fit_buffer, axis=0)

    


    newwarp=polynomial_fit(warped_binary,Minv, l_fit_mean, r_fit_mean, ploty)
    lane_curverad, Veh_pos=curvature_cal(warped_binary)
    # Combine the result with the original image
    result = cv2.addWeighted(undistort_img, 1, newwarp, 0.3, 0)
    if Veh_pos > 0:
        car_pos_text = "{:04.2f}cm right of center".format(Veh_pos*100)
    else:
        car_pos_text = '{:04.3f}cm left of center'.format(abs(Veh_pos)*100)

    cv2.putText(result, "Lane curve: {:04.2f}m".format(lane_curverad), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5,
                color=(255, 255, 255), thickness=2)
    cv2.putText(result, "Car is {}".format(car_pos_text), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color=(255, 255, 255),
                thickness=2)

    return result

FILTER_SIZE = 20
old_img_lines = None
l_fit_buffer = None
r_fit_buffer = None
det_img=process_image(original_img)

f, (ax1, ax2)=plt.subplots(1,2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(det_img)
ax2.set_title('Detected Lane lines', fontsize=30)
plt.savefig('../output_images/test6')




## Pipeline (Video)

In [1]:
# importing some useful packages
#from moviepy.editor import VideoFileClip
import imageio
imageio.plugins.ffmpeg.download()
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import cv2
import os
from moviepy.editor import VideoFileClip
import glob

## Apply Camera calibration, distortion correction
def undistortion(img):
# Arrays to store object points and image points from all the images.
    objpoints = np.load('objpoints.npy') # 3d points in real world space
    imgpoints = np.load('imgpoints.npy') # 2d points in image plane.
    shape=tuple(np.load('shape.npy'))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints,shape,None, None)
    undist=cv2.undistort(img, mtx, dist, None, mtx)
    return undist


thresh_grad_min = 20
thresh_grad_max = 200
## Define Color & gradients function
def color_threshold(img, thresh_grad_min, thresh_grad_max):
    hls=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel=hls[:,:,2]
    h_channel=hls[:,:,0]
    l_channel=hls[:,:,1]
    gray=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hsv=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    ## Sobelx
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    
    ## Sobely
    sobely = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1) # Take the derivative in y
    abs_sobely = np.absolute(sobely) # Absolute x derivative to accentuate lines away from horizontal
    mag=np.sqrt(sobelx**2+sobely**2)
    scaled_sobel = np.uint8(255*mag/np.max(mag))
    # Threshold xy gradient
    sxybinary = np.zeros_like(scaled_sobel)
    sxybinary[(scaled_sobel >= thresh_grad_min) & (scaled_sobel <= thresh_grad_max)] = 1
    
    # detecting yellows using hsv color space
    yellow_low=np.array([0,100,100], np.uint8)
    yellow_high=np.array([50, 255, 255], np.uint8)
    mask_yellow=cv2.inRange(hsv, yellow_low, yellow_high)
    
    # detecting white using hsv color space
    white_low=np.array([20, 0, 200], np.uint8)
    white_high=np.array([255, 80, 255], np.uint8)
    mask_white=cv2.inRange(hsv, white_low, white_high)
    mask=cv2.bitwise_or(mask_yellow, mask_white)
    
    # detecting edge using color threshold in s_channel
    #color_thre_min=145
    #color_thre_max=200
    #s_binary=np.zeros_like(s_channel)
    #s_binary[(s_channel>=color_thre_min)&(s_channel<color_thre_max)]=1
    # detecting shadowness and brightness change
    color_thre_min=145
    color_thre_max=200
    lab=cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    b_channel=lab[:,:,2]
    b_binary=np.zeros_like(b_channel)
    b_binary[(b_channel>=color_thre_min)&(s_channel<color_thre_max)]=1
    
    
    # stack each channel to an image to see the contribution of individual channels
    #color_binary = np.dstack(( np.zeros_like(sxybinary), sxybinary, mask)) * 255
    # combine three binary thresholds
    combined_binary=np.zeros_like(sxybinary)
    combined_binary[(sxybinary==1)|(mask==1)|(b_binary==1)]=1
    return  combined_binary


## Perspective transform
## Define the function for image unwarp
def warped_img(img):
    gray=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_size=(gray.shape[1],gray.shape[0])
    #src=np.float32([[500,500],[780,500],[img.shape[1],img.shape[0]],[150,img.shape[0]]])
    src=np.float32([[580,460],[700,460],[1040,680],[260,680]])
    dst = np.array([[200, 0], [1000, 0], [1000, img.shape[0]], [200, img.shape[0]]], np.float32)
# Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
# Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size)
    return warped, M, Minv

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image




## Define the function of detecting lane lines

def find_lines(binary):
    # calculate the current base leftx and rightx pixels location
    image_height=binary.shape[0]
    image_width=binary.shape[1]
    histogram=np.sum(binary[binary.shape[0]//2:,:], axis=0)
    midpoint=np.int(binary.shape[1]//2)
    leftx_base=np.argmax(histogram[:midpoint])           # define the base left lanes location
    rightx_base=np.argmax(histogram[midpoint:])+midpoint  # define the base right lanes location
    leftx_current=leftx_base
    rightx_current=rightx_base
    out_img=np.dstack((binary, binary, binary))
    # nonzero x & y locations
    nonzerox=np.array(binary.nonzero()[1])
    nonzeroy=np.array(binary.nonzero()[0])
    # left and right lane lines indices
    leftx_ind, rightx_ind=[],[]
    # Hyper parameters for windows
    nwindow=9
    margin=100
    minpxl=50
    
    for window in range(nwindow):
        # define the parameters of windwos on image
        win_height=np.int(image_height//nwindow)
        win_y_low=image_height-(window+1)*win_height
        win_y_high=image_height-window*win_height
        win_left_low=leftx_current-margin
        win_left_high=leftx_current+margin
        win_right_low=rightx_current-margin
        win_right_high=rightx_current+margin
        
        # identify the nonzero pixels within the window
        detected_leftx_ind=((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_left_low) &  (nonzerox < win_left_high)).nonzero()[0]
        
        detected_rightx_ind=((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_right_low) &  (nonzerox < win_right_high)).nonzero()[0]
        
        if len(detected_leftx_ind)>minpxl:
            leftx_current=np.int(np.mean(nonzerox[detected_leftx_ind]))
        if len(detected_rightx_ind)>minpxl:
            rightx_current=np.int(np.mean(nonzerox[detected_rightx_ind]))
        leftx_ind.append(detected_leftx_ind)
        rightx_ind.append(detected_rightx_ind)
    try:
        leftx_ind=np.concatenate(leftx_ind)
        rightx_ind=np.concatenate(rightx_ind)
    except ValueErrors:
        
        pass
    
    leftx=nonzerox[leftx_ind]
    rightx=nonzerox[rightx_ind]
    lefty=nonzeroy[leftx_ind]
    righty=nonzeroy[rightx_ind]
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary.shape[0]-1, binary.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    return left_fitx, right_fitx, ploty

# define polynomial function of lane lines

def polynomial_fit(binary, Minv, left_fitx, right_fitx, ploty):

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (binary.shape[1], binary.shape[0])) 
    

    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]
    #out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    return newwarp

def curvature_cal(binary):
    # Find our lane pixels first
    #leftx, rightx, ploty = find_lines(binary)
    # covert x, y from pixels to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    image_height=binary.shape[0]
    image_width=binary.shape[1]
    histogram=np.sum(binary[binary.shape[0]//2:,:], axis=0)
    midpoint=np.int(binary.shape[1]//2)
    leftx_base=np.argmax(histogram[:midpoint])           # define the base left lanes location
    rightx_base=np.argmax(histogram[midpoint:])+midpoint  # define the base right lanes location
    lane_center=np.int((leftx_base+rightx_base)//2)
    img_center=np.int(image_width//2)
    Veh_pos=np.float32((img_center-lane_center)*xm_per_pix)  # calculate the vehicel position, left '-', right off center '+'
    
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)

    # Generate x and y values for plotting
    y_eval=np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    lane_curverad=(left_curverad+right_curverad)/2

    return lane_curverad, Veh_pos



In [2]:

def process_image(img):
    # print out some info about this image
    #img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # transform from color to gray scale
    #b,g,r=cv2.split(img)
    #img=cv2.merge([r,g,b])
    img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY) #mpimg loads image as RGB
    # Apply Gaussian smoothing 
    #kernel_size=5
    #blur_gray=cv2.GaussianBlur(img_gray,(kernel_size,kernel_size),0,0)
    # Undistort the image

    undistort_img=undistortion(img)
    #color & gradients threshold
    img_warped, perspective_M, Minv=warped_img(undistort_img)
    warped_binary=color_threshold(img_warped, thresh_grad_min, thresh_grad_max)
    vertices = np.array([[(120, 710),(120, 0), (1150, 0), (1200,710)]], dtype=np.int32)
    masked_binary = region_of_interest(warped_binary, vertices)
    left_fitx, right_fitx, ploty=find_lines(masked_binary)
    
    global l_fit_buffer
    global r_fit_buffer
    global old_img_lines

    if old_img_lines is None:
        old_img_lines = masked_binary

    ret = cv2.matchShapes(old_img_lines, masked_binary, 1, 0.0)
    print(ret)
    if ret < 35:
        old_img_lines = masked_binary

        if l_fit_buffer is None:
            l_fit_buffer = np.array([left_fitx])

        if r_fit_buffer is None:
            r_fit_buffer = np.array([right_fitx])

        l_fit_buffer = np.append(l_fit_buffer, [left_fitx], axis=0)[-FILTER_SIZE:]
        r_fit_buffer = np.append(r_fit_buffer, [right_fitx], axis=0)[-FILTER_SIZE:]
        l_fit_mean=np.mean(l_fit_buffer[-15:], axis=0)
        r_fit_mean=np.mean(r_fit_buffer[-15:], axis=0)
    else:
        # Compute the mean
        l_fit_mean = np.mean(l_fit_buffer, axis=0)
        r_fit_mean = np.mean(r_fit_buffer, axis=0)

    


    newwarp=polynomial_fit(warped_binary,Minv, l_fit_mean, r_fit_mean, ploty)
    lane_curverad, Veh_pos=curvature_cal(warped_binary)
    # Combine the result with the original image
    result = cv2.addWeighted(undistort_img, 1, newwarp, 0.3, 0)
    if Veh_pos > 0:
        car_pos_text = "{:04.2f}cm right of center".format(Veh_pos*100)
    else:
        car_pos_text = '{:04.3f}cm left of center'.format(abs(Veh_pos)*100)

    cv2.putText(result, "Lane curve: {:04.2f}m".format(lane_curverad), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5,
                color=(255, 255, 255), thickness=2)
    cv2.putText(result, "Car is {}".format(car_pos_text), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color=(255, 255, 255),
                thickness=2)

    return result

In [ ]:
FILTER_SIZE = 20
old_img_lines = None
l_fit_buffer = None
r_fit_buffer = None
# read an image from a folder
#img=mpimg.imread('solidYellowCurve2.jpg')
img=cv2.imread('../test_images/test6.jpg')
b,g,r=cv2.split(img)
img=cv2.merge([r,g,b])
original_img=img
# print out some info about this image
#img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print ('This image is:', type(img),'with dimensions of',img.shape)

# transform from color to gray scale
img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY) #mpimg loads image as RGB
result=process_image(img)
f, (ax1, ax2)=plt.subplots(1,2, figsize=(24,9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(result)
ax2.set_title('Undistorted Image', fontsize=30)

In [3]:
FILTER_SIZE = 20
old_img_lines = None
l_fit_buffer = None
r_fit_buffer = None
from IPython.display import HTML
white_output = 'project_result.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)  

#HTML("""
#<video width="960" height="540" controls>
#  <source src="{0}">
#</video>
#""".format(white_output))


0.0
[MoviePy] >>>> Building video project_result.mp4
[MoviePy] Writing video project_result.mp4


  0%|                                                                                         | 0/1261 [00:00<?, ?it/s]

0.0


  0%|                                                                                 | 1/1261 [00:00<14:34,  1.44it/s]

47.8886298419681


...

0.7519464263133666


100%|█████████████████████████████████████████████████████████████████████████████▉| 1259/1261 [14:57<00:01,  1.40it/s]

1.3438165487634137


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [14:58<00:00,  1.40it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_result.mp4 

Wall time: 14min 58s
